In [1]:
import os

from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.models import Input, Model, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.callbacks import EarlyStopping
% matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

Using TensorFlow backend.


In [2]:
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def keras_discriminator(images):
  """
      Translation of tutorial into Keras.
  """
  inputs = Input(shape=images.shape[1:])
  layer = Conv2D(filters=32, kernel_size=(5, 5), activation="relu")(inputs)
  layer = MaxPool2D()(layer)
  layer = Conv2D(filters=64, kernel_size=(5, 5), activation="relu")(layer)
  layer = MaxPool2D()(layer)

  layer = Flatten()(layer)
  layer = Dense(units=512, activation="relu")(layer)
  outputs = Dense(units=10, activation="softmax")(layer)

  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

  return model

In [4]:
def train_model(images_train, labels_train):
    discriminator = keras_discriminator(images_train)
    history = discriminator.fit(x=mnist.train.images.reshape((mnist.train.images.shape[0], 28, 28, 1)),
                                y=np_utils.to_categorical(mnist.train.labels),
                                batch_size=100,
                                epochs=10,
                                callbacks=[EarlyStopping(patience=2)],
                                validation_split=0.2)
    return (discriminator, history)
    

In [5]:
def discriminator(images_train, labels_train):
    """
        Check if model files exist. If so, load the model.
        Otherwise, train the model and save it for later use.
    """
    DIRECTORY = "./learned_model/"
    MODEL_FILE = "discriminator.h5"
    
    model_file_path = os.path.join(DIRECTORY, MODEL_FILE)
    if (os.path.isfile(model_file_path)):
        discriminator = load_model(model_file_path)
    else:
        discriminator, history = train_model(images_train, labels_train)
        if not os.path.exists(DIRECTORY):
            os.makedirs(DIRECTORY)
        discriminator.save(model_file_path)
    
    return discriminator

In [6]:
images_train = mnist.train.images.reshape((mnist.train.images.shape[0], 28, 28, 1))
labels_train = np_utils.to_categorical(mnist.train.labels)
model = discriminator(images_train, labels_train)

In [7]:
err = model.evaluate(x=mnist.test.images.reshape((mnist.test.images.shape[0], 28, 28, 1)),
                 y=np_utils.to_categorical(mnist.test.labels))

 9984/10000 [============================>.] - ETA: 0s

In [8]:
print("error rate: ", 1 - err[-1])

error rate:  0.0134


In [9]:
def output_size(w, k, p, s):
  """
      Compute the expected output size given
      w = width/height of input
      k = width/height of scan
      p = padding
      s = stride
  """
  return (w - k + 2 * p) / s + 1

In [10]:
output_size(28, 2, 0, 2) # After first pooling.

14.0

In [11]:
output_size(14, 2, 0, 2) # After second pooling.

7.0